## **Recenzje artykułów**

**"Can We Do Better Than Random Start? The Power of Data Outsourcing
"** - *YI CHEN, JING DONG, XIN T. TONG*

https://arxiv.org/pdf/2205.08098.pdf

W artykule zostały przedstawione 3 algorytmy służące do znajdowania punktów startowych do rozpoczęcia algorytmów uczenia. Są nimi: Sampling-based Initial Point Selection (SIPS), Optimization-based Initial Point Selection (OIPS) oraz OIPS-SAO. Losowe wybór punktów niesie za sobą ryzyka, np. utknięcie w lokalnym minimum i nie znalezienie globalnego. Opracowane na podstawie statystycznych twierdzeń algorytmy sprawdzają się dużo lepiej. Autorzy przedstawiają teoretyczne uzasadnienie, jak i przykładowy test z wizualizacją, które potwierdzają ich skuteczność. Jednakże, zostały one potwierdzone przy spełnieniu ustalonych przez nich założeń, podczas gdy w rzeczywistości sytuacje nie są tak klarowne. Zaprezentowane algorytmy mają działać na małej próbce danych by koszt ich użycia był niewielki. Problem został połączony z data outsourcingiem, w którym ilość danych otrzymanych do pracy jest odgórnie narzucona.


**"It takes a lot of energy for machines to learn – here’s why AI is so power-hungry"** - *Kate Saenko*

https://theconversation.com/it-takes-a-lot-of-energy-for-machines-to-learn-heres-why-ai-is-so-power-hungry-151825

Artykuł porusza problem zużycia energi przy trenowaniu modeli bazujących na samouczeniu. Autorka stawia tezę dyskredytującą aktualne techniki trenowania modelu, uważając je za nieefektywne. Porównując je z działaniem mózgu człowieka, przytacza przykład modelu BERT analizującego język, w którym maszyna potrzebowała kilka tysięcy razy większej ilości danych oraz przejścia po nich kilkadziesiąt razy by otrzymać zadawalające rezultaty. Wyliczono, że model ten, przetwarzając 3.3 miliarda słów z książek i artykułów z Wikipedii, miał ślad węglowy porównywalny z przelotem Boeinga 747 z Nowego Yorku do San Francisco i z powrotem.

 Autorka przedstawia również kolejną szkodliwą dla środowiska technikę rozbudowywania modeli do ogromnych rozmiarów, dającą dodatkowy procent skuteczności, mimo tego, że większość neuronów jest w tym modelu nieaktywnych.

W przyszłości jest szansa na zmiejszynie zużycia energetycznego dzięki opracowaniu lepszych algorytmów i bardziej optymalnych metod uczenia.

**Interpretability Beyond Feature Attribution: Quantitative Testing with Concept Activation Vectors (TCAV)**

*Been Kim, Martin Wattenberg, Justin Gilmer, Carrie Cai, James Wexler, Fernanda Viegas, Rory Sayres*

https://arxiv.org/abs/1711.11279

W artykule wprowadzono pomysł wyjaśnień opartych na konceptach. Potrzebne do tego są dwa zbiory, jeden zawierający przykłady konceptu, drugi który nie zawiera przykładów konceptu. Aby uzyskać Concept Activation Vector (CAV), z modelu wyłuskujemy wartości aktywacji z dowolnej warstwy. Wyznaczamy liniową granicę decyzyjną pomiędzy zbiorami w takiej reprezentacji. CAV to wektor prostopadły do tej granicy.

Zalety:
+ w czasie projektowania i treningu sieci nie musimy wyznaczać konceptów. Definiuje je użytkownik końcowy tworząc dwa zbiory przykładów
+ wg autorów wyjaśnienia uzyskane w ten sposób są bardziej zrozumiałe dla ludzi

Wady:
+ skuteczność metody zależy od wyboru zbiorów przykładów
+ podatność na manipulacje poprzez komponowanie odpowiednich zbiorów przykładów

## The data randomization test

### 1.Wymieszanie etykiet

In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!jar xvf challenge-nr-1-test.zip

In [ ]:
import requests 
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torchinfo import summary

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

dataset = torchvision.datasets.ImageFolder('../content/challenge-nr-1-test/', transform=transform)

### 2.Budowa modelu

In [ ]:
len_train_dataset=2500
len_test_dataset=500
data_test, data_train = torch.utils.data.random_split(dataset, [len_test_dataset, len_train_dataset])
test_dataloader  = torch.utils.data.DataLoader(dataset=data_test, batch_size=16, shuffle=True)
train_dataloader  = torch.utils.data.DataLoader(dataset=data_train, batch_size=16, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

resnet18 = models.resnet18(pretrained=True)
resnet18.fc=nn.Linear(512, 3)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = resnet18.to(device)
params_to_update = model_ft.parameters()
optimizer = optim.AdamW(params_to_update, lr=0.001, weight_decay=0.25)
criterion = nn.CrossEntropyLoss()
n_epoch = 3


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:

train_results = []
test_results = []
for epoch in range(n_epoch):
        print('Epoch {}/{}'.format(epoch + 1, n_epoch))
        print('-' * 10)

        model_ft.train()
        train_loss = 0
        train_correct = 0
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model_ft(inputs)
            loss = criterion(outputs, labels)
            train_loss += loss.item()
            train_correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
            loss.backward()
            optimizer.step()

        model_ft.eval()
        test_loss = 0
        test_correct = 0
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model_ft(inputs)
            batch_loss = criterion(outputs, labels).item()
            with torch.no_grad():
                test_loss += batch_loss
                test_correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

        test_acc = test_correct / len_test_dataset
        train_acc = train_correct / len_train_dataset
        train_results += [{'loss': train_loss, 'acc': train_acc}]
        test_results += [{'loss': test_loss, 'acc':test_acc}]



Epoch 1/3
----------
Epoch 2/3
----------
Epoch 3/3
----------


### 3.Wyjaśnienia

In [ ]:
for i in [10,1000,2000]:
    images, labels = data_test[i]
    plt.imshow(images.cpu().permute(1, 2, 0))
    plt.axis("off")
    print(labels)
    plt.show()

In [ ]:
from captum.attr import KernelShap
for i in [10,1000,2000]:
  X, label = data_test[i]
  X = X.double()
  mask = segmentation.quickshift(
          X.permute(1, 2, 0), 
          kernel_size=14, 
          max_dist=7, 
          ratio=0.5
        )
  exp_ks = KernelShap(model.double())

  exp_ks = exp_ks.attribute(
          X.unsqueeze(0),
          target=label, 
          n_samples=200, 
          feature_mask=torch.as_tensor(mask),
          show_progress=True
        )
  imgshow(X,label)
  show_attr(exp_ks[0])

## The model parameter randomization test